In [6]:
# !micromamba install torch
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 19.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 34.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.3 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 24.5 MB/s eta 0:00:00m eta 0:00:010:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 49.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 134.2/158.2 MB 78.0 MB/s eta 0:00:01ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [ ]:
# import torch
import cv2
from groundingdino.util.inference import load_model, predict
from segment_anything import sam_model_registry, SamPredictor
import matplotlib.pyplot as plt

# --- Load Grounding DINO ---

WEIGHTS_PATH = "/home/niall/weights/"
dino_config = "cs231n/config/GroundingDINO_SwinT_OGC.py"
dino_weights = "groundingdino_swint_ogc.pth"
dino_model = load_model(dino_config, WEIGHTS_PATH+dino_weights)

# --- Load SAM ---

sam_checkpoint = "sam_vit_h_4b8939.pth"
sam = sam_model_registry["vit_b"](checkpoint=WEIGHTS_PATH+sam_checkpoint)
predictor = SamPredictor(sam)

# --- Load image ---

image_path = "example.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# --- Grounding DINO: Zero-shot detection ---

prompt = "screen, monitor"
boxes, logits, phrases = predict(
    model=dino_model,
    image=image_rgb,
    caption=prompt,
    box_threshold=0.35,
    text_threshold=0.25
)

# --- SAM: Segment using the boxes ---

predictor.set_image(image_rgb)
masks = []
for box in boxes:
    mask, _, _ = predictor.predict(box=box.numpy(), multimask_output=False)
    masks.append(mask)

# --- Visualize results ---

for mask in masks:
    image_rgb[mask] = (0, 255, 0)  # green overlay

for box in boxes:
    x0, y0, x1, y1 = box
    cv2.rectangle(image_rgb, (int(x0), int(y0)), (int(x1), int(y1)), (255, 0, 0), 2)

plt.imshow(image_rgb)
plt.title("Zero-shot Detection + Segmentation")
plt.axis("off")
plt.show()


final text_encoder_type: bert-base-uncased
